In [1]:
import sys
sys.path.append('..')
from speckle import VirtExp
import speckle
import numpy as np
import math
import bpy
import mathutils
import os

In [2]:
output_folder = r"D:\Experiment Quality\Correlation test\T_joint_Rob"
pattern_path = os.path.join(output_folder, "im.tiff")
normals_path = os.path.join(output_folder, "grad.tiff")
model_path = os.path.join(output_folder, "model.blend")
output_path = os.path.join(output_folder, "render_0_0.tiff")
output_path2 = os.path.join(output_folder, "render_0_1.tiff")
calib_path = os.path.join(output_folder, "calibration.caldat")
mesh_path = os.path.join(output_folder, "T_joint_3D.mesh")

In [21]:
image_size = (2000, 3500) # doesnt matter
target_size = (20, 20)
pixel_size_physical = 0.00345  # mm
imaging_dist = 165
#focal_length = 47
#M = focal_length / (imaging_dist - focal_length)
M = 0.6
focal_length = (M * imaging_dist)/(1 + M)
pixel_size = pixel_size_physical/M
speckle_size = 4.0 * pixel_size
output_DPI = 2500
n_samples = 500

In [38]:
print(speckle_size)
imaging_dist = 165
M = 0.6
focal_length = (M * imaging_dist)/(1 + M)
#focal_length = 60
#M = focal_length / (imaging_dist - focal_length)
#print(M)

0.0346468085106383
0.5714285714285714


In [34]:
pat1 = speckle.SpeckleImage(image_size, speckle_size)
pat1.set_physical_dim(target_size, speckle_size, output_DPI)
im1 = pat1.gen_pattern()
pat1.pattern_gradient()
pat1.im_save(pattern_path)
# Generate normals map
normals_map = speckle.SpeckleImage(pat1.image_size, 60, normal_map_filter=1)
normals_map.gen_pattern()
normals_map.pattern_gradient()
normals_map.generate_norm_map(binary_map=pat1.gradient)
normals_map.grad_save(normals_path)

In [4]:
# Generate blender scene
a = VirtExp(pattern_path, normals_path, output_path, model_path,
            objects_position="fixed")
# Set up the scene
# Get default properties
p = a.get_default_params()
# Add the default target
target = a.add_FEA_part(mesh_path, thickness=0.005, solidify_flag=False)
target.location = (0, 0, -0.150)
target.rotation_mode = 'QUATERNION'
a.rotate_around_z(target, 180)
#target = a.add_rect_target((0.1, 0.1, 0.002))

# Add the light panel
# Calculate desired orientation of the light
light_target = np.array([0, 0, 0.005])
light_target_orient = p["light_target"] - np.array(p["light_pos"])
# Calculate the rotation angle of the light
light_angle = a.calc_rot_angle(p["light_init_rot"], 
                                  light_target_orient)
a.add_light(p["light_type"], pos=p["light_pos"], orient=light_angle,
               energy=p["light_energy"], spot_size=p["light_spotsize"],
               spot_blend=p["light_spot_blend"], 
               shadow_spot_size = p["light_shad_spot"])

# CAMERAS
# Add straight camera
# Calculate desired orientation of the cam
cam0_pos = np.array([0, 0, 0.165]) 
cam0_target_orient = p["cam0_target"] - cam0_pos
cam0_target_dist = np.linalg.norm(cam0_target_orient)
cam_angle = a.calc_rot_angle(p["cam_init_rot"], cam0_target_orient)
# Define distortions of the lenses
cam0_k = [-0.2183, 13.6610, 1.8567, -0.002477, 0.008435, None, None]
#cam0_k = [0, 0, 0, 0, 0, None, None]
cam0 = a.add_camera(pos=cam0_pos, orient=cam_angle, 
                    fstop=p["cam_fstop"], 
                    focal_length=focal_length,
                    obj_distance=cam0_target_dist,
                    k1=cam0_k[0], k2=cam0_k[1], k3=cam0_k[2], 
                    p1=cam0_k[3], p2=cam0_k[4], c0=cam0_k[5], c1=cam0_k[6])  
# a.rotate_around_z(cam0, 0.1)

# Add cross camera
ang_y = math.radians(12.0)
cam1_pos = np.array([math.sin(ang_y), 0, math.cos(ang_y)]) * imaging_dist*1e-3
cam1_target_orient = p["cam1_target"] - cam1_pos
cam1_target_dist = np.linalg.norm(cam1_target_orient)
cam_angle = a.calc_rot_angle(p["cam_init_rot"], cam1_target_orient)
# Define distortions of the lenses
cam1_k = [-0.2233, 11.4379, 0.6723, -0.003311, 0.01129, None, None]
#cam1_k = [0, 0, 0, 0, 0, None, None]
cam1 = a.add_camera(pos=cam1_pos, orient=cam_angle, 
                    fstop=p["cam_fstop"],
                    focal_length=focal_length,
                    obj_distance=cam1_target_dist,
                    k1=cam1_k[0], k2=cam1_k[1], k3=cam1_k[2], 
                    p1=cam1_k[3], p2=cam1_k[4], c0=cam1_k[5], c1=cam1_k[6])  
# a.rotate_around_z(cam1, -0.5)

# Define the material and assign it to the cube
a.add_material(target)
# Write the calibration file
a.generate_calib_file(cam0, cam1, calib_path)
# Set the renderer up and render image
a.set_renderer(cam0, n_samples=n_samples)
# Save the model
a.save_model()

Info: Saved "model.blend"


## Depth of field study

### Render front camera and cross camera using different f-stops

In [22]:
render_folder = os.path.join(output_folder, 
                             f"ang{math.degrees(ang_y):1g}")
if not os.path.exists(render_folder):
    os.mkdir(render_folder)  
fnum_list = [4, 5.6, 8, 11, 16, 22, 32]
calib_path = os.path.join(render_folder, 'calibration.caldat')
a.generate_calib_file(cam0, cam1, calib_path)
for fstop in fnum_list:
    cam0.data.dof.aperture_fstop = fstop
    cam1.data.dof.aperture_fstop = fstop
    def_path = os.path.join(render_folder, f'Image_fstop_{int(10*fstop):d}_0.tiff')
    a.set_renderer(cam0, n_samples=n_samples)
    a.render_scene(def_path)
    a.add_image_distortion(cam0, def_path)
    def_path = os.path.join(render_folder, f'Image_fstop_{int(10*fstop):d}_1.tiff')
    a.set_renderer(cam1, n_samples=n_samples)
    a.render_scene(def_path)
    a.add_image_distortion(cam1, def_path)

Error: Render error (No error) cannot save: 'D:\Experiment Quality\Correlation test\T_joint_Rob\ang12\Image_fstop_40_0.tiff'


RuntimeError: Error: Render error (No error) cannot save: 'D:\Experiment Quality\Correlation test\T_joint_Rob\ang12\Image_fstop_40_0.tiff'
